### Funcion: get_features_cat_regression (Unai)

Esta función recibe como argumentos un dataframe, el nombre de una de las columnas del mismo (argumento 'target_col'), que debería ser el target de un hipotético modelo de regresión, es decir debe ser una variable numérica continua o discreta pero con alta cardinalidad y una variable float "pvalue" cuyo valor por defecto será 0.05.

La función debe devolver una lista con las columnas categóricas del dataframe cuyo test de relación con la columna designada por 'target_col' supere en confianza estadística el test de relación que sea necesario hacer (es decir la función debe poder escoger cuál de los dos test que hemos aprendido tiene que hacer).

La función debe hacer todas las comprobaciones necesarias para no dar error como consecuecia de los valores de entrada. Es decir hará un check de los valores asignados a los argumentos de entrada y si estos no son adecuados debe retornar None y printar por pantalla la razón de este comportamiento. Ojo entre las comprobaciones debe estar que "target_col" hace referencia a una variable numérica continua del dataframe.

In [11]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency
from sklearn.preprocessing import OrdinalEncoder

def get_features_cat_regression(df, target_col, p_value=0.01):
    if df.empty:
        print("El dataframe esta vacío")
        return None
    if not pd.api.types.is_numeric_dtype(df[target_col]):
        print("La columna que has puesto no es una columna numerica")
        return None
    if not isinstance(p_value, float) or 0 > p_value or 1 < p_value:
        print("El p_value no tiene un valor valido, recuerda que tiene que estar entre 0 y 1")
        return None
    if target_col not in df:
        print("La columna no esta en el Dataframe, cambiala por una valida")
        return None
    
    categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
    
    #ordinal_encoder = OrdinalEncoder(categorical_columns)

    encoded_df = pd.get_dummies(df_titanic, columns=categorical_columns, dtype=float)
    encoded_columns = encoded_df.columns
    new_categorical_columns = [col for col in encoded_columns if col.startswith(tuple(categorical_columns))]

    print(encoded_df.head())

    #categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

    #if df['survived'].shape[0]<30:
    significant_columns = []
    for col in new_categorical_columns:
        if df['survived'].shape[0]<30:
            t_statistic, p_value_t = stats.ttest_ind(encoded_df[col], encoded_df[target_col], nan_policy='omit')
            if p_value_t < p_value:
                significant_columns.append([col, t_statistic, p_value_t])
        else:
            z_statistic = (encoded_df[col].mean() - encoded_df[target_col].mean()) / (encoded_df[col].std() / np.sqrt(len(encoded_df)))
            print(np.exp(stats.norm.cdf(abs(z_statistic))))
            p_value_z = 2 * (1 - np.exp(stats.norm.cdf(abs(z_statistic))))
            if p_value_z < p_value:
                significant_columns.append([col, z_statistic, p_value_z])
                
    print(significant_columns)
    significant_categorical_cols = [col[0].split('_')[0] for col in significant_columns ]
    significant_categorical_cols = list(set(significant_categorical_cols))


    return significant_categorical_cols
    
df_titanic = pd.read_csv('./data/titanic.csv')
target_col='survived'
print(target_col)
get_features_cat_regression(df_titanic,target_col)

survived
   survived  pclass   age  sibsp  parch     fare  adult_male  alone  \
0         0       3  22.0      1      0   7.2500        True  False   
1         1       1  38.0      1      0  71.2833       False  False   
2         1       3  26.0      0      0   7.9250       False   True   
3         1       1  35.0      1      0  53.1000       False  False   
4         0       3  35.0      0      0   8.0500        True   True   

   sex_female  sex_male  ...  deck_C  deck_D  deck_E  deck_F  deck_G  \
0         0.0       1.0  ...     0.0     0.0     0.0     0.0     0.0   
1         1.0       0.0  ...     1.0     0.0     0.0     0.0     0.0   
2         1.0       0.0  ...     0.0     0.0     0.0     0.0     0.0   
3         1.0       0.0  ...     1.0     0.0     0.0     0.0     0.0   
4         0.0       1.0  ...     0.0     0.0     0.0     0.0     0.0   

   embark_town_Cherbourg  embark_town_Queenstown  embark_town_Southampton  \
0                    0.0                     0.0      

['sex', 'class', 'deck', 'embark', 'alive', 'who', 'embarked']

In [8]:


from scipy.stats import mannwhitneyu
import pandas as pd
import numpy as np
from scipy import stats

def get_features_cat_regression(df, target_col, p_value=0.05):
    if df.empty:
        print("El dataframe esta vacío")
        return None
    if not pd.api.types.is_numeric_dtype(df[target_col]):
        print("La columna que has puesto no es una columna numerica")
        return None
    if not isinstance(p_value, float) or 0 > p_value or 1 < p_value:
        print("El p_value no tiene un valor valido, recuerda que tiene que estar entre 0 y 1")
        return None
    if target_col not in df:
        print("La columna no esta en el Dataframe, cambiala por una valida")
        return None
    
    categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
    
    relevant_columns = []
    
    for col in categorical_columns:
        grouped = df.groupby(col)[target_col].apply(list).to_dict()
        f_vals = []
        for key, value in grouped.items():
            f_vals.append(value)
        f_val, p_val = stats.f_oneway(*f_vals)
        if p_val <= p_value:
            relevant_columns.append([col, f_val, p_val])

    return relevant_columns

df_titanic = pd.read_csv('./data/titanic.csv')
target_col='survived'
print(target_col)
get_features_cat_regression(df_titanic,target_col)

survived


c:\Users\unais\anaconda3\envs\DataScience\Lib\site-packages\scipy\stats\_stats_py.py:4167: ConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(stats.ConstantInputWarning(msg))


[['sex', 372.4057236022147, 1.406066130879677e-69],
 ['embarked', 13.605270445693582, 1.514339014290716e-06],
 ['class', 57.964817590910116, 2.1832474151179777e-24],
 ['who', 207.65379810794602, 1.0340021305330169e-74],
 ['embark_town', 13.605270445693582, 1.514339014290716e-06],
 ['alive', inf, 0.0]]